## Importing the dataset

In [1]:
import numpy as np
import pandas as pd

In [2]:
training_set = pd.read_csv('ml-100k/u1.base', delimiter = '\t')
training_set = np.array(training_set, dtype = 'int')
test_set = pd.read_csv('ml-100k/u1.test', delimiter = '\t')
test_set = np.array(test_set, dtype = 'int')

In [3]:
print(training_set)

[[        1         2         3 876893171]
 [        1         3         4 878542960]
 [        1         4         3 876893119]
 ...
 [      943      1188         3 888640250]
 [      943      1228         3 888640275]
 [      943      1330         3 888692465]]


## Getting total number of users and movies

In [4]:
total_movies = int(max(max(training_set[:,1]), max(test_set[:, 1])))
total_users = int(max(max(training_set[:,0]), max(test_set[:, 0])))
print('total movies: ', total_movies)
print('total users: ', total_users)

total movies:  1682
total users:  943


## Data Preprocessing

In [5]:
def preprocessing(data):
    new_data = []
    for user in range(1, total_users + 1):
        data_movies = data[:, 1][data[:, 0] == user]
        data_ratings = data[:, 2][data[:, 0] == user]
        ratings = np.zeros(total_movies)
        ratings[data_movies - 1] = data_ratings
        new_data.append(list(ratings))
    return new_data

updated_training_set = preprocessing(training_set)
updated_test_set = preprocessing(test_set)

In [7]:
import torch
updated_training_set = torch.FloatTensor(updated_training_set)
updated_test_set = torch.FloatTensor(updated_test_set)

## Architecture of AutoEncoder and initializing optimizer

In [8]:
import torch.nn as nn
import torch.optim as optim

In [9]:
class SAE(nn.Module):
    def __init__(self, ):
        super(SAE, self).__init__()
        self.fc1 = nn.Linear(total_movies, 25)
        self.fc2 = nn.Linear(25, 12)
        self.fc3 = nn.Linear(12, 25)
        self.fc4 = nn.Linear(25, total_movies)
        self.activation = nn.Sigmoid()
        
    def forward(self, x):
        x = self.activation(self.fc1(x))
        x = self.activation(self.fc2(x))
        x = self.activation(self.fc3(x))
        x = self.fc4(x)
        return x
    
sae = SAE()
criterion = nn.MSELoss()
optimizer = optim.RMSprop(sae.parameters(), lr = 0.01, weight_decay = 0.5)

## Training and Calculating the Training Loss

In [10]:
from torch.autograd import Variable

In [11]:
epochs = 200
for epoch in range(1, epochs + 1):
    train_loss = 0
    s = 0.
    for user in range(total_users):
        input = Variable(updated_training_set[user]).unsqueeze(0)
        target = input.clone()
        if torch.sum(target.data > 0 ) > 0:
            output = sae(input)
            target.require_grad = False
            output[target == 0] = 0
            loss = criterion(output, target)
            mean_corrector = total_movies/float(torch.sum(target.data > 0)+ 1e-10)
            loss.backward()
            train_loss  += np.sqrt(loss.data * mean_corrector)
            s += 1.
            optimizer.step()
    print('Epoch: '+str(epoch)+ ' Loss: '+str(train_loss/s))

Epoch: 1 Loss: tensor(1.6873)
Epoch: 2 Loss: tensor(1.0829)
Epoch: 3 Loss: tensor(1.0494)
Epoch: 4 Loss: tensor(1.0380)
Epoch: 5 Loss: tensor(1.0327)
Epoch: 6 Loss: tensor(1.0294)
Epoch: 7 Loss: tensor(1.0276)
Epoch: 8 Loss: tensor(1.0262)
Epoch: 9 Loss: tensor(1.0250)
Epoch: 10 Loss: tensor(1.0248)
Epoch: 11 Loss: tensor(1.0241)
Epoch: 12 Loss: tensor(1.0237)
Epoch: 13 Loss: tensor(1.0231)
Epoch: 14 Loss: tensor(1.0228)
Epoch: 15 Loss: tensor(1.0229)
Epoch: 16 Loss: tensor(1.0226)
Epoch: 17 Loss: tensor(1.0223)
Epoch: 18 Loss: tensor(1.0219)
Epoch: 19 Loss: tensor(1.0218)
Epoch: 20 Loss: tensor(1.0212)
Epoch: 21 Loss: tensor(1.0197)
Epoch: 22 Loss: tensor(1.0167)
Epoch: 23 Loss: tensor(1.0152)
Epoch: 24 Loss: tensor(1.0142)
Epoch: 25 Loss: tensor(1.0096)
Epoch: 26 Loss: tensor(1.0109)
Epoch: 27 Loss: tensor(1.0068)
Epoch: 28 Loss: tensor(1.0072)
Epoch: 29 Loss: tensor(1.0032)
Epoch: 30 Loss: tensor(1.0025)
Epoch: 31 Loss: tensor(0.9992)
Epoch: 32 Loss: tensor(0.9976)
Epoch: 33 Loss: t

## Testing and Calculating the Test Loss

In [13]:
test_loss = 0
s = 0.
for user in range(total_users):
  input = Variable(updated_training_set[user]).unsqueeze(0)
  target = Variable(updated_test_set[user]).unsqueeze(0)
  if torch.sum(target.data > 0) > 0:
    output = sae(input)
    target.require_grad = False
    output[target == 0] = 0
    loss = criterion(output, target)
    mean_corrector = total_movies/float(torch.sum(target.data > 0) + 1e-10)
    test_loss += np.sqrt(loss.data * mean_corrector)
    s += 1.
print('test loss: '+str(test_loss/s))

test loss: tensor(0.9551)
